<a href="https://colab.research.google.com/github/adventHymnals/resources/blob/master/adventHymnals_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Section 1
You'll need to expand to interact with interactive cells.

Installation of pdflatext takes quite a while

In [ ]:
%%capture
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
os.environ['ROOT_DIR']="adventhymnals_v4"
os.environ['PRIVATE_DIR']="private"


import getpass
password = getpass.getpass("Please enter ssh zip password: ")
os.environ['SSH_ZIP_PASSWORD'] = password



####
#### sheets
####
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

file_path = os.environ['ROOT_DIR'] + '/public/Hymnals'
print(file_path)

#worksheet = gc.open(file_path, folder_id="1uXjDtbXRe78EhnCzOe8gjk-gXOdyPNsh").sheet1
worksheet = gc.open("Hymnals").sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
#print(rows)

# Convert to a DataFrame and render.
#import pandas as pd
#pd.DataFrame.from_records(rows)

hymnals = [row[0] + '-v4' for row in rows[1:]]

# Convert to YAML format
yaml_string = "hymnals:\n"
for hymnal in hymnals:
    yaml_string += f"  - '{hymnal}'\n"

#print(yaml_string)

# Save to a file named 'hymnals.yaml'
file_path = '/tmp/hymnals.yaml'
with open(file_path, 'w') as file:
    file.write(yaml_string)

print(f"YAML file saved as '{file_path}'")


### process configs
worksheet = gc.open("configs").sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
# print(rows)
for row in rows:
    key, value = row
    os.environ[key] = value


Please enter ssh zip password: ··········


In [ ]:
%%capture
%%shell
apt-get install xattr > /dev/null
sudo apt-get install texlive-latex-base
sudo apt-get install texlive-fonts-recommended
sudo apt-get install texlive-fonts-extra


sshZipped="/content/drive/MyDrive/$ROOT_DIR/$PRIVATE_DIR/ssh.zip"
cp $sshZipped /tmp/
unzip -P "$SSH_ZIP_PASSWORD" -oq /tmp/ssh.zip
rm -rf /root/.ssh
mkdir -p /root/.ssh
rsync -aq ssh/  ~/.ssh/
rsync -aq ssh/   /root/.ssh/
ssh-keyscan GitHub.com > /root/.ssh/known_hosts #2>&1 >/dev/null
ssh-keyscan GitHub.com > ~/.ssh/known_hosts #2>&1 >/dev/null
chmod 644 ~/.ssh/known_hosts
chmod 600 ~/.ssh/id_rsa
chmod 644 /root/.ssh/known_hosts
chmod 600 /root/.ssh/id_rsa

#eval "$(ssh-agent)"
#ssh-add ~/.ssh/id_rsa
#echo "$SSH_ZIP_PASSWORD" | ssh-add ~/.ssh/id_rsa

eval "$(ssh-agent)"
echo "$SSH_ZIP_PASSWORD" | ssh-add ~/.ssh/id_rsa
rm -rf hymnals_repo
git clone git@github.com:adventHymnals/hymnals_repo.git
mv /tmp/hymnals.yaml hymnals_repo

git config --global user.email "$GitCommitEmail"
git config --global user.name "$GitCommitUser"
cd hymnals_repo
# echo "$(date)" >> tmp
git add . && git commit -m "$(date)" && git push origin master
cd ..


cd hymnals_repo
eval "$(ssh-agent)"
echo "$SSH_ZIP_PASSWORD" | ssh-add ~/.ssh/id_rsa
#rm -rf hymnals_repo
rm -rf tmps
mkdir -p tmps
cat hymnals.yaml | grep -o "'[^\']*'" | sed "s/'//g" | while read -r hymnal; do
    echo "=>$hymnal"
    csycms_hymnal=$(echo "$hymnal"|sed -e 's/v4$/csycms/')
    echo $csycms_hymnal

    cd tmps
    git clone "git@github.com:adventHymnals/$hymnal.git"
    ls "$hymnal"
    cd "$hymnal"

    if git rev-parse HEAD >/dev/null 2>&1; then
      echo "Repository is not empty."
    else ## if _v4 repo does not exist
      currentPath=$(pwd)
      cd /tmp
      rm -rf "$csycms_hymnal"
      git clone "git@github.com:adventHymnals/$csycms_hymnal.git"
      cd "$csycms_hymnal"
      rm -rf .git .github
      echo "ls of $csycms_hymnal::"

      # rename /chapter.md
      find "./" -wholename "*/chapter.md"  | while read -r line ; do
        newPath=$( echo $line | sed -e 's|/chapter\.md|/index.md|' )
        mv "$line" "$newPath"
      done

      # rename /docs.md
      find "./" -wholename "*/docs.md"  | while read -r line ; do  ## remove also trailing comma from file names
          newPath=$( echo $line | sed -e 's|/\([^\]*\)/docs\.md|/\1.md|' | sed -e 's|,\.md|.md|')
          # echo "$line==>$newPath"
          mv "$line" "$newPath"
      done

      if [ -d "./indices" ]; then # if its a hymnal, (with indices dir)
          mv "./index.md" "$hymnal/preface.md"
          mv "./indices/index.md" "$hymnal/index.md"
          rm -rf "./indices"
      fi

      find  "./" -type d -empty -delete
      #ls -lha ./
      cd ..
      rsync -av "$csycms_hymnal/" "$currentPath/"
      cd "$currentPath"
      git add . && git commit -m "$(date)" && git push origin master
      #ls -lha
    fi
    cd ..
    cd ..
done

Create the _v4 hymnal repos
No means of filtering spreadsheets by directory, so it is important that spreadsheets don't share titles
https://docs.gspread.org/en/latest/api/client.html#gspread.Client.open

In [ ]:
%%capture
%%shell
cd /content/hymnals_repo
cd tmps

for hymnal in *v4*; do
    #echo "=>$hymnal"
    drive_hymnal=$(echo "$hymnal"|sed -e 's/-v4$//')
    #echo "<==$csycms_hymnal"
    if [ -d "/content/drive/MyDrive/$ROOT_DIR/public/Hymnals/$drive_hymnal" ]; then
      echo "Folder exists!"
    else
      echo "Folder does not exist!"
      #// do the conversion here...


      source_folder="./$hymnal"  # Replace with the path to the source folder
      output_folder="/content/drive/MyDrive/$ROOT_DIR/public/Hymnals/$drive_hymnal"  # Replace with the path to the output folder

      # Create the output folder if it doesn't exist
      mkdir -p "$output_folder"

      # Function to convert .md files to .docx
      convert_to_docx() {
        local source_file="$1"
        local output_file="$2"
        pandoc "$source_file" -o "$output_file"
      }

      # Find all .md files in the source folder and its subfolders
      find "$source_folder" -type f -name "*.md" |
      while IFS= read -r md_file; do
      relative_path="${md_file#$source_folder/}"
        #echo $relative_path
        # Determine the output file path by replacing the .md extension with .docx in the output folder
        docx_file="${md_file%.md}.docx"
        output_file="$output_folder/${relative_path%.md}.docx"
        #output_file="$output_folder/${docx_file##*/}"
        mkdir -p "$(dirname "$output_file")"
        echo "==>$output_file"

        # Convert .md to .docx and save it in the output folder
        file_name="$md_file"
        #cat $file_name
        sed -i '/^#/d' $file_name
        ## Format the title. Remove hymn number and hymnal name
        sed -i 's/^\([ ]*\)[0-9]*[\.]*[ ]*\(.*\) \- [^\-]*$/\1\2/' $file_name
        sed -i 's/^\-\-$/---/g' $file_name
        sed -i '/^\^/d' $file_name
        ## sed -i '/```/{:a;N;/```/!ba;s/```.*$/```/2;}' $file_name
        new_file_name="new_file_name"
        ## Remove everything after the last ```
        #cat $file_name
        grep -n -m 2 '```' $file_name | tail -n 1 | cut -d ':' -f 1 | xargs -I {} head -n {} $file_name > "$new_file_name"
        #sed -i 's/\(```.*```\).*/\1/' $file_name
        #cat $new_file_name
        mv $new_file_name $file_name

       convert_to_docx "$md_file" "$output_file"
       #cat $new_file_name
      done

    fi
done

Sync github repo with our docs here.
If folder does not exist in drive, run pandoc to convert every file to docx and save in drive, otherwise

Run this cell atleast 3 times until the only output is of the hymnal dir names. If an error occurs, you will have to rename the directory causing the error manually

In [ ]:
%%shell

cd /content/hymnals_repo
cd tmps

# if matched..
for hymnal in *; do
    echo "=>$hymnal"
    drive_hymnal=$(echo "$hymnal"|sed -e 's/-v4$//')
    #echo "<==$csycms_hymnal"



      source_folder="./$hymnal"  # Replace with the path to the source folder
      output_folder="/content/drive/MyDrive/$ROOT_DIR/public/Hymnals/$drive_hymnal"  # Replace with the path to the output folder

      # Find all .md files in the source folder and its subfolders
      find "$output_folder" -type f -name "*.docx" |
      while IFS= read -r file_path; do

        filename=$(basename "$file_path")
        pattern="^[0-9]{2,3}\..*\.docx"
        if [[ $filename =~ $pattern ]]; then
            folder_name=$(dirname "$file_path")
            folder_number=$(echo "$folder_name" | sed 's/.*\.\([0-9]*\).*/\1/' | sed 's/^[0]*//' )
            file_number=$(echo "$file_path" | sed 's/.*\/\([0-9]*\)\..*/\1/' | sed 's/^[0]*//')
            result=$((folder_number + file_number - 1))
            fileNumber=$(printf "%04d" "$result")
            if [ "$fileNumber" = "0000" ]; then
              echo "Variable is 0000."
              fileNumber=""
              newFileName=$(echo "$filename" | sed 's/\([0-9]*\)\(\..*\)/\2/' |sed 's/\.//')
            else
              newFileName=$(echo "$filename" | sed 's/\([0-9]*\)\(\..*\)/\2/')
            fi

            #newFileName=$(echo "$filename" | sed 's/\([0-9]*\)\(\..*\)/\2/')
            newFileName="$fileNumber$newFileName"
            newFileName="$folder_name/$newFileName"
            echo "=>$newFileName"
            mv "$file_path" "$newFileName"
            # echo "$newFileName"
        #else
            #echo "Filename does not match the pattern."
        fi
      done


      find "$output_folder" -type f -name "*.docx" |
      while IFS= read -r file_path; do

        filename=$(basename "$file_path")
        folder_name=$(dirname "$file_path")
        #tmp=$(dirname $tmp)
        #echo $tmp | sed 's/[0-9]*\.\([^\.]*\.[^\.]*$\)/\1/'
        newDir=$(echo $folder_name | sed 's/[0-9]*\.\([^\.]*$\)/\1/') # move last dir, then go down the chain
        #echo "$folder_name=>$newDir"
        #mv "$folder_name" "$newDir"
        if [ -d "$folder_name" ] && [ "$folder_name" != "$newDir" ]; then
          #echo "Directory '$directory_name' exists."
          echo "$folder_name=>$newDir"
          mv "$folder_name" "$newDir"
        #else
          #echo "Directory '$directory_name' does not exist."
        fi
      done





done

# move...

=>campus-melodies-v4
=>christ-in-song-v4
=>church-hymnal-v4
=>Hymns-for-Gods-Peculiar-People-v4
=>hymns-for-the-poor-of-the-flock-v4
=>millenial-harp-v4
=>nyimbo-za-kristo-v4
=>seventh-day-adventist-hymnal-v4
=>wende-nyasaye-v4


## Section 2

In [ ]:
%%shell
eval "$(ssh-agent)"
echo "$SSH_ZIP_PASSWORD" | ssh-add ~/.ssh/id_rsa
cd /content/hymnals_repo
cd tmps

# if matched..
for hymnal in *; do
    echo "=>$hymnal"
    drive_hymnal=$(echo "$hymnal"|sed -e 's/-v4$//')
    #echo "<==$csycms_hymnal"



      source_folder="./$hymnal"  # Replace with the path to the source folder
      output_folder="/content/drive/MyDrive/$ROOT_DIR/public/Hymnals/$drive_hymnal"  # Replace with the path to the output folder

      rm -rf "/tmp/$drive_hymnal"
      mkdir -p "/tmp/$drive_hymnal"
      echo "" > "$output_folder/fullhymnal.md"

      echo "[Download](/$hymnal/$hymnal.pdf)" > /tmp/index2
      echo " " >> /tmp/index2
      echo "[Full Hymnal](/$hymnal/$hymnal.html)" >> /tmp/index2
      echo " " >> /tmp/index2
      echo "|Songs|" >> /tmp/index2
      echo "|------|" >> /tmp/index2

      find "$output_folder" -type f -name "*.docx" | sort |
      while IFS= read -r file_path; do

        filename=$(basename "$file_path")
        #echo $file_path
        pattern="^[0-9]{4}\..*\.docx"
        if [[ $filename =~ $pattern ]]; then
            #fid = getoutput("xattr -p 'user.drive.id' $file_path")

            fid=$(xattr -p 'user.drive.id' "$file_path")
            fid=$(echo "$fid" | sed 's/-/%2D/g')
            #echo "fid$fid"
            pandoc $file_path -s -o "/tmp/$drive_hymnal/tmp.md"
            file_number=$(echo "$filename" | sed 's/\([0-9]*\)\..*/\1/' | sed 's/^[0]*//')
            #echo $file_number

            ## sed -i "s/title:[ ]*'\(.*\)',/## \1/g" "$output_folder/fullhymnal.md"
            # remove ' if both first and last chars are ', eg in: ‘Delightful day! first gift of Heaven’
            sed -i "s/title:\(.*\)/## $file_number. \1\n[edit](https:\/\/docs.google.com\/document\/d\/$fid\/edit?mode=html)\n/g" "/tmp/$drive_hymnal/tmp.md"
            sed  -i "s/\-\-\-//g" "/tmp/$drive_hymnal/tmp.md"
            cat "/tmp/$drive_hymnal/tmp.md" >> "$output_folder/fullhymnal.md"
            echo " " >> "$output_folder/fullhymnal.md"
            echo " " >> "$output_folder/fullhymnal.md"

            tmpPath=$(echo $file_path | sed 's/.*Hymnals\///' |sed 's/\.docx/.md/')
            tmpDir=$(dirname $tmpPath)
            #rm -rf "/tmp/$drive_hymnal/$tmpDir"
            mkdir -p "/tmp/$drive_hymnal/$tmpDir"
            tmpPathtmpPath=$(basename $tmpPath | sed 's/^[0-9]*\.//' | sed 's/://g')
            cat "/tmp/$drive_hymnal/tmp.md" >> "/tmp/$drive_hymnal/$tmpDir/$tmpPathtmpPath"
            tmpPath="$tmpDir/$tmpPathtmpPath"
            numberAndTitle=$(grep -o '##.*' "/tmp/$drive_hymnal/$tmpPath" | sed 's/## *//')
            hrefPath_=$(echo $file_path | sed 's/.*Hymnals\///' |sed 's/\.docx/.html/' | sed 's|^[^/]*/||') #| sed 's|^[^/]*/||'
            hrefPath=$(dirname $hrefPath_)
            hrefFile=$(basename  $hrefPath_| sed 's/^[0-9]*\.//')
            #echo $hrefPath
            hrefPath="/$hymnal/$hrefPath/$hrefFile"
            #echo "$hrefPath"
            echo "|[$numberAndTitle]($hrefPath)|" >> /tmp/index2
        fi
      done
      rm "/tmp/$drive_hymnal/tmp.md"
      echo "" >> /tmp/index2
      cp "$output_folder/fullhymnal.md" "/tmp/$drive_hymnal/$drive_hymnal/$hymnal.md"
      cat /tmp/index2 > "/tmp/$drive_hymnal/$drive_hymnal/index.md"
      #cat "$output_folder/fullhymnal.md" >> "/tmp/$drive_hymnal/tmp.md"
      #cat "/tmp/$drive_hymnal/tmp.md" > "$output_folder/fullhymnal.md"
      #rm "/tmp/$drive_hymnal/tmp.md"
      pandoc "$output_folder/fullhymnal.md" -o "$output_folder/fullhymnal.docx"
      pandoc --toc "$output_folder/fullhymnal.docx" -o "$output_folder/fullhymnal.pdf"
      cp "$output_folder/fullhymnal.pdf" "/tmp/$drive_hymnal/$drive_hymnal/$hymnal.pdf"
      #mv "/tmp/$drive_hymnal/$tmpDir/" "/tmp/$drive_hymnal/$tmpDir-bac/"
      cd "./$hymnal"
      cp -r .git "/tmp/$drive_hymnal/$drive_hymnal/"
      cd ..
      currentDir=$(pwd)
      cd "/tmp/$drive_hymnal/$drive_hymnal/"
      git branch gh-pages ## inscase of an error, don't destroy master branch of main repo
      git checkout gh-pages
      #find "./" -type d -not -name '.*' -exec rm -r {} +
      #rsync -av "/tmp/$drive_hymnal/$drive_hymnal/" "./"  ## For some reason "/tmp/$drive_hymnal/$tmpDir/*" with the trailing slash alone
      #touch .nojekyll
      git add . && git commit -m "$(date)" && git push origin gh-pages -f
      #cd ..
      cd "$currentDir"
done

# move...

# Saving data to Google Sheets


This snippet uses the open-source [gspread](https://github.com/burnash/gspread) library for interacting with Sheets.

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

sh = gc.create('A new spreadsheet')

# Open our new sheet and add some data.
worksheet = gc.open('A new spreadsheet').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)
# Go to https://sheets.google.com to see your new spreadsheet.

# Importing data from Google Sheets

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('Hymnals').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
pd.DataFrame.from_records(rows)